# COGS 108 - Final Project 

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

* [  ] YES - make available
* [ x ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Yang Li
- Yiou Lyu
- Linfeng Hu
- Ruby Celeste Marroquin 

# Group Members IDs

- A15560579
- A15930345
- A15473121
- A16094382

# Research Question

How does the regional economic status of each province in mainland China correlate to its breakout and recovery of COVID-19?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

(Copy this information for each dataset)
- Dataset Name: 
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [ ]:
import pandas as pd
import json
import codecs
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import patsy
import scipy.stats as stats

import bs4
from bs4 import BeautifulSoup

# Data Cleaning

In [ ]:
#with codecs.open('Data/virus.json', 'r', 'utf-8') as data_file:
    #data_teacher = json.load(data_file, 'utf-8')

#topic[worksheet] = data_teacher[worksheetID]['Topic']
#out = codecs.open('Worksheet.csv', 'w', 'utf-8')
#out.write(topic[worksheet])

To clean our data, our general approach is to represent datasets in pandas dataframe. Then we drop irrelevant information or outliers in data. We also rename the columns to make it easier for later analyses.

Firstly, we deal with the datasets that consist of economic status data. 

This is the income per capita value for each province in mainland China. Income is measured in yuan. 

In [ ]:
Income = pd.read_csv('Data/Income.csv')
Income = Income.dropna(axis=1, how='all')
Income.head()

This is the per capita Gross Regional Product value for each province. GRP per capita is measured in yuan.

In [ ]:
GRP = pd.read_csv('Data/GRP.csv')
GRP = GRP.dropna(axis=1, how='all')
GRP.head()

Next, we move on to clean the population density related datasets.

Population per province here is calculated in the unit of 10000 persons). It includes all residents (permanent and temporary, rural and urban)at the end of that year.

In [ ]:
population = pd.read_csv('Data/Population.csv')
population = population.dropna(axis = 1, how = 'all')
population.head()

To calculate population density of a region, we also need to areas of each province. Here, area of each province is measured in unit of square kilometers.

Since we only need the area information of each separate region, we will drop the "Toal" row at the end which contains information about the total area of China(judging by the data contained, the row name should be a typo). We will also drop the proportion row because we only need the area number. 

In [ ]:
area = pd.read_csv('Data/Area.csv')
area = area.dropna(axis = 1, how = 'all')
#shorten column names to make following analysis simpler
area = area.rename(columns={"Area (sq.km)": "Area"})
area = area[area.District != 'Toal']
area = area[area.columns[:2]]
area.head()

we also need to remove "," in the string in order to change it to type int.

In [ ]:
def remove_comma(strin):
    strin = strin.replace(',','')
    return strin

area['Area']= area['Area'].apply(remove_comma)
area.head()

In [ ]:
# read virus data into dataframes 

list_of_virus_data = list()

# append data between Feb 1 and Feb 25 to list
for i in range(20200201,20200226): 
    path = './Data/virus/' + str(i) + '.csv'
    df = pd.read_csv(path)
    df.headers = path
    list_of_virus_data.append(df)
    
# File 20200226.csv is missing, reason unknow. 

    
# append data between Feb 27 and Feb 29 to list
for i in range(20200227,20200230): 
    path = './Data/virus/' + str(i) + '.csv'
    df = pd.read_csv(path)
    df.headers = path
    list_of_virus_data.append(df)

# append data between Mar 1 and  Mar 1 to list
for i in range(20200301,20200302): 
    path = './Data/virus/' + str(i) + '.csv'
    list_of_virus_data.append(pd.read_csv(path))
    
print('number of dataframes for virus: ',len(list_of_virus_data))

# access ith elment in the list using list_of_virus_data[i]
# for example list_of_virus_data[0] gives the first dataframe


## Start cleaning virus data

### Clean 0th to 1th df in the list 

In [ ]:
# Clean 0th to 1th df in the list 
for i in range(0,2):
    # get the df of the ith day
    df = list_of_virus_data[i]
    # use the first data row as column names
    df.columns = df.iloc[0]
    # drop first row, because is was used as header
    df = df.drop(0)
    # drop the column '1', because it is irrlavent
    df = df.drop(1, axis=1)
    # save cleaned data to list_of_virus_data 
    list_of_virus_data[i] = df

### Clean 2th df in the list

In [ ]:
# Clean 2th df in the list
# get the df of the ith day
df = list_of_virus_data[2]
# reset column names
df.columns = ["Province/Region/City", "Confirmed Cases", 1]
# drop meaningless 1" column,  keep "Confirmed Cases" and "Province/Region/City"
df = df.drop(1, axis=1)
# Drop the last row, because it is comment instaed of data
df = df.drop(df.shape[0] - 1)
# save cleaned data to list_of_virus_data 
list_of_virus_data[2] = df

### Clean 3th to 10th df in the list 

In [ ]:
# Clean 3th to 10th df in the list 
for i in range(3,11):
    # get the df of the ith day
    df = list_of_virus_data[i]
    # use the first data row as column names
    df.columns = df.iloc[0]
    # drop first row, because is was used as header
    df = df.drop(0)
    # drop the column '1', because it is irrlavent
    df = df.drop(1, axis=1)
    # save cleaned data to list_of_virus_data
    list_of_virus_data[i] = df

In [ ]:
df1 = list_of_virus_data[1]
df2 = list_of_virus_data[2]
df1.merge(df2,how='left')

# Data Analysis & Results

Correlation between Population density and economic status
This part is to analyze the correlation between population density and economic status. We found the data for both population and area by each province. It needs to be calculated to population density first by dividing population by area.

We decide to use the population data from the most recent year available, which is 2018.
We also need to change the column name to "district" to correspond with the area dataset.

In [ ]:
population_2018 = population[population.columns[:2]]
population_2018 = population_2018.rename(columns = {'Region':'District'})
population_2018.head()

we will now merge the population and area dataset by the province name. We choose to "inner" merge them because if there is a province with either no population or area, we are unable to calculate the population density.
Then we could have calculate the population density by dividing population by area.

In [ ]:
popu_density = pd.merge(population_2018,area,on = 'District')
popu_density['Area'] = pd.to_numeric(popu_density['Area'])
popu_density['2018'] = pd.to_numeric(popu_density['2018'])
popu_density['population density'] = popu_density['2018']/popu_density['Area']
popu_density = popu_density.drop(columns = 'Area')
popu_density = popu_density.drop(columns = '2018')
popu_density.head()


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*